In [43]:
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [44]:
data = pd.read_csv('properties.csv')
data.head()

,title,location,spec,price,description,features
0,Newly Built 4 Bedroom Terraced Duplex,Ikate Lekki Lagos,4 Beds | 4 Baths | 5 Toilets,"20,000,000/year",Newly built 4 bedroom terrace duplex \n ...,"['Street Lights', 'Big Compound', '24 Hours Se..."
1,3 Bedroom Apartment,Oniru Victoria Island Lagos,3 Beds | 3 Baths | 4 Toilets,"20,000,000/year",CW07369 \n This spacious 3-bedroom apar...,NaN
2,2 Bedroom Apartment,Lekki Phase 1 Lekki Lagos,2 Beds | 2 Baths | 3 Toilets,"13,000,000/year",CW07377 This 2-bedroom apartment delivers a si...,NaN
3,3 Bedroom Apartment,Ikoyi Lagos,3 Beds | 3 Baths | 4 Toilets,"20,000,000/year",CW07192 \n \n 3-BEDROOM APARTME...,NaN
4,3 Bedroom Apartment With Bq,Old Ikoyi Ikoyi Lagos,3 Beds | 3 Baths | 4 Toilets,"30,000,000/year",CW07241 \n \n 3 BEDROOM APARTME...,NaN


### Data Cleaning ###

In [45]:
df = data.copy()

# Check for missing values
def check_missing(df):
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing Percentage': missing_percent
    })
    return round(missing_df[missing_df['Missing Count'] > 0], 2)

check_missing(df)

,Missing Count,Missing Percentage
spec,3131,22.04
features,9711,68.36


In [46]:
def clean_price(price):
    if pd.isnull(price):
        return np.nan
    price = str(price).lower().replace('/year', '').strip()
    price = re.sub(r'[^\d]', '', price)
    return float(price) if price else np.nan

def clean_spec(spec):
    if pd.isnull(spec):
        return {'bedrooms': np.nan, 'bathrooms': np.nan, 'toilets': np.nan}
    
    parts = str(spec).split('|')
    cleaned = {'bedrooms': np.nan, 'bathrooms': np.nan, 'toilets': np.nan}
    
    for part in parts:
        part = part.strip().lower()
        match = re.search(r'(\d+)', part)
        
        if match:
            number = int(match.group(1))
            if 'bed' in part:
                cleaned['bedrooms'] = number
            elif 'bath' in part:
                cleaned['bathrooms'] = number
            elif 'toilet' in part:
                cleaned['toilets'] = number
    
    return cleaned

def clean_location(location):
    if pd.isnull(location):
        return np.nan
    
    location = str(location).strip()
    parts = location.split()
    
    if len(parts) < 2:
        return location.title()
    
    area = ' '.join(parts[:-1])
    return area.title()

def extract_all_features(row):
    """Extract features from both description AND features column"""
    
    features = {
        'has_pool': 0,
        'has_gym': 0,
        'has_parking': 0,
        'has_bq': 0,
        'has_elevator': 0,
        'is_newly_built': 0,
        'is_furnished': 0,
        'is_serviced': 0,
        'has_security': 0,
        'has_generator': 0,
        'has_ac': 0,
        'has_wifi': 0
    }
    
    # Check description
    desc = str(row['description']).lower() if pd.notna(row['description']) else ''
    
    if any(word in desc for word in ['pool', 'swimming']):
        features['has_pool'] = 1
    if 'gym' in desc or 'fitness' in desc:
        features['has_gym'] = 1
    if 'parking' in desc or 'garage' in desc:
        features['has_parking'] = 1
    if 'bq' in desc or 'boys quarter' in desc:
        features['has_bq'] = 1
    if 'elevator' in desc or 'lift' in desc:
        features['has_elevator'] = 1
    if 'newly' in desc or 'brand new' in desc:
        features['is_newly_built'] = 1
    if 'furnished' in desc:
        features['is_furnished'] = 1
    if 'serviced' in desc or 'service charge' in desc:
        features['is_serviced'] = 1
    if 'security' in desc:
        features['has_security'] = 1
    if 'generator' in desc:
        features['has_generator'] = 1
    if 'air condition' in desc or ' ac ' in desc:
        features['has_ac'] = 1
    if 'wifi' in desc or 'internet' in desc:
        features['has_wifi'] = 1
    
    # Check features column (override if found there)
    feat_str = str(row['features']).lower() if pd.notna(row['features']) else ''
    
    if 'pool' in feat_str or 'swimming' in feat_str:
        features['has_pool'] = 1
    if 'gym' in feat_str or 'fitness' in feat_str:
        features['has_gym'] = 1
    if 'parking' in feat_str or 'garage' in feat_str:
        features['has_parking'] = 1
    if 'security' in feat_str:
        features['has_security'] = 1
    if 'generator' in feat_str or 'power' in feat_str:
        features['has_generator'] = 1
    if 'air condition' in feat_str or 'ac' in feat_str:
        features['has_ac'] = 1
    if 'wifi' in feat_str or 'internet' in feat_str:
        features['has_wifi'] = 1
    
    return pd.Series(features)

def get_property_type(title):
    if pd.isnull(title):
        return 'Other'
    
    title = str(title).lower()
    
    if 'duplex' in title or 'storey' in title:
        return 'Duplex'
    elif 'terrace' in title:
        return 'Terrace'
    elif 'room self contained' in title or 'room self contain' in title or 'roomself' in title or 'room self' in title or 'self contained' in title or 'self contain' in title:
        return 'Single Room'
    elif 'shop' in title:
        return 'Shop'
    elif 'detached' in title and 'semi' not in title:
        return 'Detached'
    elif 'semi' in title or 'semi-detached' in title:
        return 'Semi-Detached'
    elif 'maisonette' in title:
        return 'Maisonette'
    elif 'townhouse' in title:
        return 'Townhouse'
    elif 'apartment' in title or 'bungalow' in title or 'bedroom' in title or 'flat' in title or 'house' in title or 'master' in title:
        return 'Apartment'
    elif 'penthouse' in title:
        return 'Penthouse'
    elif 'warehouse' in title:
        return 'Warehouse'
    elif 'villa' in title:
        return 'Villa'
    elif 'office' in title:
        return 'Office'
    elif 'commercial' in title:
        return 'Commercial'
    else:
        return 'Other'

In [47]:
# Apply the cleaning functions to the dataframe
df['price_clean'] = df['price'].apply(clean_price)

spec_df = df['spec'].apply(clean_spec).apply(pd.Series)
df['bedrooms'] = spec_df['bedrooms']
df['bathrooms'] = spec_df['bathrooms']
df['toilets'] = spec_df['toilets']

df['area'] = df['location'].apply(clean_location)

feature_df = df.apply(extract_all_features, axis=1)
df = pd.concat([df, feature_df], axis=1)

df['property_type'] = df['title'].apply(get_property_type)

duplicates = df.duplicated(subset=['title', 'price_clean', 'area']).sum()
print(f"Duplicates removed: {duplicates}")
df = df.drop_duplicates(subset=['title', 'price_clean', 'area'])

df.head(20)

Duplicates removed: 5102


,title,location,spec,price,description,features,price_clean,bedrooms,bathrooms,toilets,...,has_bq,has_elevator,is_newly_built,is_furnished,is_serviced,has_security,has_generator,has_ac,has_wifi,property_type
0,Newly Built 4 Bedroom Terraced Duplex,Ikate Lekki Lagos,4 Beds | 4 Baths | 5 Toilets,"20,000,000/year",Newly built 4 bedroom terrace duplex \n ...,"['Street Lights', 'Big Compound', '24 Hours Se...",20000000.0,4.0,4.0,5.0,...,0,0,1,0,0,1,0,1,1,Duplex
1,3 Bedroom Apartment,Oniru Victoria Island Lagos,3 Beds | 3 Baths | 4 Toilets,"20,000,000/year",CW07369 \n This spacious 3-bedroom apar...,NaN,20000000.0,3.0,3.0,4.0,...,1,0,0,0,0,1,0,0,0,Apartment
2,2 Bedroom Apartment,Lekki Phase 1 Lekki Lagos,2 Beds | 2 Baths | 3 Toilets,"13,000,000/year",CW07377 This 2-bedroom apartment delivers a si...,NaN,13000000.0,2.0,2.0,3.0,...,0,0,0,0,0,0,0,0,0,Apartment
3,3 Bedroom Apartment,Ikoyi Lagos,3 Beds | 3 Baths | 4 Toilets,"20,000,000/year",CW07192 \n \n 3-BEDROOM APARTME...,NaN,20000000.0,3.0,3.0,4.0,...,0,0,1,0,1,0,0,0,0,Apartment
4,3 Bedroom Apartment With Bq,Old Ikoyi Ikoyi Lagos,3 Beds | 3 Baths | 4 Toilets,"30,000,000/year",CW07241 \n \n 3 BEDROOM APARTME...,NaN,30000000.0,3.0,3.0,4.0,...,1,1,0,0,1,1,0,1,0,Apartment
5,Furnished 2 Bedroom Apartment,Oniru Victoria Island Lagos,2 Beds | 2 Baths | 3 Toilets,"30,000,000/year",CW07199 \n \n 2 BED FURNISHED A...,NaN,30000000.0,2.0,2.0,3.0,...,0,0,0,1,1,0,0,0,1,Apartment
6,3 Bedroom Maisonette,Banana Island Ikoyi Lagos,3 Beds | 3 Baths | 4 Toilets,"35,000,000/year",CW07233 \n \n 3 BED MAISONETTE ...,NaN,35000000.0,3.0,3.0,4.0,...,1,0,0,0,1,1,0,0,0,Maisonette
7,Multiple Office,Old Ikoyi Ikoyi Lagos,NaN,"220,000/sqm",CW07201 \n \n Multiple Office| ...,NaN,220000.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,Office
8,Commercial Property,Victoria Island Lagos,NaN,"100,000,000/year",CW07283 \n \n COMMERCIAL PROPER...,NaN,100000000.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,Commercial
9,Commercial Property,Victoria Island Lagos,NaN,"80,000,000/year",CW07368 \n \n Commercial Proper...,NaN,80000000.0,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,0,Commercial


In [48]:
check_missing(df)

,Missing Count,Missing Percentage
spec,2048,22.50
features,6155,67.62
bedrooms,2415,26.53
bathrooms,2232,24.52
toilets,2111,23.19


In [49]:
df.describe()

,price_clean,bedrooms,bathrooms,toilets,has_pool,has_gym,has_parking,has_bq,has_elevator,is_newly_built,is_furnished,is_serviced,has_security,has_generator,has_ac,has_wifi
count,9.103000e+03,6688.000000,6871.000000,6992.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000
mean,3.005068e+07,2.561453,2.651288,3.186928,0.131825,0.110074,0.292102,0.187081,0.057893,0.189608,0.106339,0.398001,0.330001,0.036142,0.209382,0.113150
std,4.094132e+08,1.319168,1.392666,1.544676,0.338319,0.312999,0.454754,0.389998,0.233554,0.392012,0.308287,0.489512,0.470239,0.186653,0.406890,0.316793
min,1.000000e+00,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.200000e+06,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000e+06,2.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.500000e+07,3.000000,4.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,2.500000e+10,10.000000,10.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
# df.value_counts('property_type')
# df.pivot_table(index='property_type', columns='bedrooms', values='title', aggfunc='count', fill_value=0)
# df[df['bedrooms'].isnull() | df['bathrooms'].isnull() | df['toilets'].isnull()].pivot_table(index='property_type', values='title', aggfunc='count', fill_value=0)
# df[df['property_type'] == 'Other']['title'].value_counts().head(20)
df['property_type'].value_counts()

property_type
Apartment        5402
Duplex           1773
Single Room       610
Other             326
Office            246
Shop              246
Commercial        117
Semi-Detached     106
Maisonette         97
Terrace            82
Detached           80
Townhouse          17
Villa               1
Name: count, dtype: int64

In [51]:
def extract_bedrooms_from_title(title):
    if pd.isnull(title):
        return np.nan
    
    patterns = [
        r'(\d+)\s*bed',
        r'(\d+)\s*br\b',
        r'(\d+)br\b',
        r'(\d+)\s*bedroom',
        r'studio',
        r'(\d+)\s*room'
    ]
    
    title_lower = str(title).lower()
    
    if 'studio' in title_lower or 'single room' in title_lower:
        return 1
    
    for pattern in patterns:
        match = re.search(pattern, title_lower)
        if match:
            if pattern == r'studio':
                return 1
            return int(match.group(1))
    return np.nan

df['bedrooms_from_title'] = df['title'].apply(extract_bedrooms_from_title)
df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms_from_title'])

residential_types = ['Apartment', 'Duplex', 'Terrace', 'Maisonette', 'Townhouse', 
                     'Detached', 'Semi-Detached', 'Villa']

mask = (df['property_type'].isin(residential_types)) & (df['bedrooms'].notna()) & (df['bathrooms'].isna())
df.loc[mask, 'bathrooms'] = df.loc[mask, 'bedrooms']

df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].median())
df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].median())
df['toilets'] = df['toilets'].fillna(df['toilets'].median())

# Office/Commercial properties don't have bedrooms - mark as 0 or drop them
non_residential = ['Office', 'Commercial', 'Other']
mask = df['property_type'].isin(non_residential)

print(f"Found {mask.sum()} non-residential properties")
df_residential = df[~mask].copy()
df_residential = df_residential.drop(columns=['bedrooms_from_title'], errors='ignore')

print("\nMissing values:")
missing = check_missing(df_residential)
print(missing)

Found 689 non-residential properties

Missing values:
          Missing Count  Missing Percentage
spec               1629               19.36
features           5654               67.20


In [52]:
# Drop unnecessary columns and save cleaned data
columns_to_drop = ['spec', 'location', 'description', 'features', 'price']
df_residential.drop(columns=columns_to_drop, inplace=True)
df_residential.to_csv('properties_cleaned.csv', index=False)